Import

In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import pickle
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from torch.utils.data import DataLoader
import torchmetrics
from pytorch_tabular import TabularModel
from ff_utils import FeedForward, test_model, MyDataset

Implementazione API

In [2]:
# Output: unique ID of the team
def getName():
    return "RACITI GABRIELE"

# Input: Test dataframe
# Output: PreProcessed test dataframe
def preprocess(df, clfName):
    if clfName.lower() == "lr":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/standardization.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        norm = pickle.load(open("./models/normalizer_l2.save", 'rb'))
        X = pd.DataFrame(norm.transform(X))
        dfNew = pd.concat([y, X], axis = 1)
        return dfNew
    elif clfName.lower() == "knr":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/minmax_scaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        pca = pickle.load(open("./models/pca70minmax.save", 'rb'))
        X = pd.DataFrame(pca.transform(X))
        dfNew = pd.concat([y, X], axis = 1)
        return dfNew
    elif clfName.lower() =="svr":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/minmax_scaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        dfNew = pd.concat([y, X], axis = 1)
        return dfNew
    elif clfName.lower() =="rf":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/minmax_scaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        pca = pickle.load(open("./models/pca30minmax.save", 'rb'))
        X = pd.DataFrame(pca.transform(X))
        dfNew = pd.concat([y, X], axis = 1)
        return dfNew
    elif clfName.lower() =="ff":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/minmax_scaler.save", 'rb'))
        X = pd.DataFrame(scaler.transform(X))
        dfNew = pd.concat([y, X], axis = 1)
        return dfNew
    elif clfName.lower() =="tb":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/standardization.save", 'rb'))
        X_scaled = scaler.transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns) 
        dfNew = pd.concat([y, X_scaled], axis=1)
        return dfNew
    elif clfName.lower() =="tf":
        X = df.drop(columns=['Year'])
        y = df['Year']
        scaler = pickle.load(open("./models/standardization.save", 'rb'))
        X_scaled = scaler.transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns) 
        dfNew = pd.concat([y, X_scaled], axis=1)
        return dfNew
    
    
          


# Input: Regressor name ("lr": Linear Regression, "SVR": Support Vector Regressor, ...)
# Output: Regressor object
def load(clfName):
    if (clfName.lower() == "lr"):
        clf = pickle.load(open("./models/linearRegression.save", 'rb'))
        return clf
    elif (clfName.lower() == "svr"):
        clf = pickle.load(open("./models/svr.save", 'rb'))
        return clf
    elif (clfName.lower() == "knr"):
        clf = pickle.load(open("./models/knn.save", 'rb'))
        return clf
    elif (clfName == "rf"):
        clf = pickle.load(open("./models/randomForest.save", 'rb'))
        return clf
    elif (clfName.lower() == "ff"):
        model = FeedForward(90, 128, 1)
        model.load_state_dict(torch.load('./models/ff.pth'))
        return model 
    elif (clfName.lower() == "tb"):
        model = TabularModel.load_model('./models/tabnet', map_location=torch.device('cpu'))
        return model
    elif (clfName.lower() == "tf"):
        model = TabularModel.load_model('./models/tabtransformer', map_location=torch.device('cpu'))
        return model
    else:
        return None
    
# Input: PreProcessed dataset, Regressor Name, Regressor Object 
# Output: Performance dictionary
def predict(df, clfName, clf):
    X = df.drop(columns=['Year'])
    y = df['Year']
    if clfName.lower() =="ff":
        # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        device = 'cpu'
        X = X.values
        y = y.values
        test_dataset = MyDataset(X,y)
        test_loader = DataLoader(test_dataset, batch_size=1)
        clf.to(device)
        labels, ypred = test_model(clf, test_loader, device)
        
        mse_metric = torchmetrics.MeanSquaredError().to(device)
        mae_metric = torchmetrics.MeanAbsoluteError().to(device)
        mape_metric = torchmetrics.MeanAbsolutePercentageError().to(device)
        r2_metric = torchmetrics.R2Score().to(device)

        mse = mse_metric(ypred, labels).item()
        mae = mae_metric(ypred, labels).item()
        mape = mape_metric(ypred, labels).item()
        r2 = r2_metric(ypred, labels).item()
    elif clfName.lower() =="tb":
        ypred = clf.predict(X)
        mse = mean_squared_error(y, ypred)
        mae = mean_absolute_error(y, ypred)
        mape = mean_absolute_percentage_error(y, ypred)
        r2 = r2_score(y, ypred)
    elif clfName.lower() =="tf":
        ypred = clf.predict(X)
        mse = mean_squared_error(y, ypred)
        mae = mean_absolute_error(y, ypred)
        mape = mean_absolute_percentage_error(y, ypred)
        r2 = r2_score(y, ypred)
    else:
        ypred = clf.predict(X)
        mse = mean_squared_error(y, ypred)
        mae = mean_absolute_error(y, ypred)
        mape = mean_absolute_percentage_error(y, ypred)
        r2 = r2_score(y, ypred)
    perf = {"mse": mse, "mae": mae, "mape": mape, "r2score": r2}
    return perf
    

Esempio esecuzione

In [3]:
# "N.B.: E' preferibile inserire le predizioni riguardanti il modello SVR per ultime a causa della lentezza di predizione dovuta alla complessità 
# del modello (circa 20 minuti per 25mila campioni sulla mia macchina). La scelta di non rendere il modello meno complesso per velocizzare le 
# predizioni è stata fatta per massimizzare i risultati del modello. In scenari reali, si potrebbe preferire una via di mezzo o utilizzare
#  modelli più veloci e performanti.

FILENAME = "test_data.csv"
CLF_NAME_LIST = ["lr", "knr", "rf", "ff", "tb", "tf", "svr"]
df = pd.read_csv(FILENAME)

#Esecuzione degli algoritmi
for modelName in CLF_NAME_LIST:
    dfProcessed = preprocess(df, modelName)
    clf = load(modelName)
    perf = predict(dfProcessed, modelName, clf)
    print("RESULT team: "+str(getName())+" algoName: "+ modelName + " perf: "+ str(perf))




RESULT team: RACITI GABRIELE algoName: lr perf: {'mse': 77.12388430135948, 'mae': 6.30986623368177, 'mape': 0.003166652333647582, 'r2score': 0.2835460075656998}
RESULT team: RACITI GABRIELE algoName: knr perf: {'mse': 74.40238744606964, 'mae': 6.269314179420018, 'mape': 0.0031452430798335, 'r2score': 0.30882776437855564}
RESULT team: RACITI GABRIELE algoName: rf perf: {'mse': 76.3739501528225, 'mae': 6.252239817342722, 'mape': 0.0031378714564361267, 'r2score': 0.290512634307199}
RESULT team: RACITI GABRIELE algoName: ff perf: {'mse': 68.86982727050781, 'mae': 5.730935573577881, 'mape': 0.0028767965268343687, 'r2score': 0.3596232533454895}


2024-07-02 11:42:38,094 - {pytorch_tabular.tabular_model:165} - INFO - Experiment Tracking is turned off

2024-07-02 11:42:38,113 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RESULT team: RACITI GABRIELE algoName: tb perf: {'mse': 68.62740126952995, 'mae': 5.7732044175034956, 'mape': 0.002898104313781601, 'r2score': 0.36247537224886894}


2024-07-02 11:42:41,097 - {pytorch_tabular.tabular_model:165} - INFO - Experiment Tracking is turned off

2024-07-02 11:42:41,097 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RESULT team: RACITI GABRIELE algoName: tf perf: {'mse': 82.50808369580339, 'mae': 6.573350120450326, 'mape': 0.0032990011221899654, 'r2score': 0.23352867263559574}
RESULT team: RACITI GABRIELE algoName: svr perf: {'mse': 70.41183485052065, 'mae': 5.53360882747824, 'mape': 0.0027802649616192458, 'r2score': 0.34589860650482196}
